In [13]:
"""
Decodes the ETROC and Oscilliscope binary
Performs fit on the KCU Clock and MCP signals
"""
import sys
sys.path.append("..")

from binary_decoders import etroc, lecroy
from oscilliscope_fitting import clock, mcp
import awkward as ak
import uproot
import importlib

: 

In [ ]:

# CLOCK CONFIGURABLES
CLOCK_THRESH_LOW, CLOCK_THRESH_HIGH = 0.25, 0.8 #used to pick out the edges (between 0 and 1, percentage of the absolute amplitude)
CLOCK_MEAUREMENT_POINT = 0.5 #between 0 and 1, after the fit, where along the fitted y axis do we take the clock value
CHANNEL_NUM = 1 #channel with square wave voltage (by index!! so subtract 1 to whatever it is on oscilloscope)


def consolidate_acquisition(output_file_path: str, etroc_binary_paths: list[str], mcp_binary_path: str, clock_binary_path: str):
    def convert_etroc_binary() -> ak.Array:
        first_binary_convert = etroc.converter(etroc_binary_paths, skip_trigger_check=True)
        return etroc.root_dumper(first_binary_convert) # root dumper name is due to history

    def convert_mcp_binary() -> ak.Array:
        return lecroy.LecroyReader(mcp_binary_path)

    def convert_clock_binary() -> ak.Array:
        return lecroy.LecroyReader(clock_binary_path)

    print("Converting ETROC Binary")
    etroc_data = convert_etroc_binary()

    print("Converting MCP Channel Binary")
    mcp_waveform = convert_mcp_binary()
    # fit mcp using dat2root stuff keep it as general "fitting" and return only the used fitting params
    # -> all LP2_XX and amp
    print("Performing fit on MCP")
    mcp_fit_data = mcp.fit(mcp_waveform)

    print("Converting Clock Channel Binary")
    clock_waveform = convert_clock_binary()
    print("Performing Clock Wavefrom Fits")
    clock_timestamps = clock.calc_clock(
        clock_waveform.x, 
        clock_waveform.y,
        CLOCK_THRESH_LOW, CLOCK_THRESH_HIGH, CLOCK_MEAUREMENT_POINT
    )

    print("Merging MCP, and clock fits")
    mcp_map = dict(zip(
        ak.fields(mcp_waveform), ak.unzip(mcp_waveform)
    ))
    mcp_fit_map = dict(zip(
        ak.fields(mcp_fit_data), ak.unzip(mcp_fit_data)
    ))
    etroc_data_map = dict(zip(
        ak.fields(etroc_data), ak.unzip(etroc_data)
    ))
    
    #NOTE !!!!!!!ORDER MATTERS!!!!!!!!!!! 
    # THE CHANNEL FIELD FOR SCOPE_DATA IS DIFFERENT THAN RECO DATA!
    # taking reco data channels because of what was done previously...
    merged_data_map = mcp_fit_map | mcp_map |  etroc_data_map
    
    merged_data_map['Clock'] = clock_timestamps

    with uproot.recreate(output_file_path) as output:
        output["pulse"] = merged_data_map

In [33]:
importlib.reload(lecroy)
importlib.reload(mcp)
import matplotlib.pyplot as plt
import numpy as np


ref_channel = lecroy.LecroyReader("/home/users/hswanson13/ETL_TestingDAQ/unit_test/input_data/run_5100/C4--Trace5100.trc")
r_trig, r_horz = ref_channel.get_segment_times()
# plt.scatter(ref_channel.x[0], ref_channel.y[0])

mcp_data = lecroy.LecroyReader("/home/users/hswanson13/ETL_TestingDAQ/unit_test/input_data/run_5100/C2--Trace5100.trc")
m_trig, m_horz = mcp_data.get_segment_times()

clock_data = lecroy.LecroyReader("/home/users/hswanson13/ETL_TestingDAQ/unit_test/input_data/run_5100/C3--Trace5100.trc")

c_trig, c_horz = clock_data.get_segment_times()

old = uproot.open(
    "/home/users/hswanson13/ETL_TestingDAQ/unit_test/asserted_output/run_5100/converted_run5100.root"
)["pulse"].arrays()

# print(old.type.show())
# print(old["timeoffsets"][:,2])
# print(c_horz - m_horz)
# print(m_horz - c_horz)
# print(c_horz - r_horz)


def generate_old_awk_array(oscilliscope_reference_reader: lecroy.LecroyReader, mcp_reader: lecroy.LecroyReader, clock_reader: lecroy.LecroyReader) -> ak.Array:
    """
    This is just so it can work with the old code (ie the dat2root compiled script). If an updated mcp fitting routine was made this code could be removed.
    5000 * {
        i_evt: uint32,
        segment_time: float32,
        channel: 4 * 502 * float32,
        time: 1 * 502 * float32,
        timeoffsets: 8 * float32 #one for each channel
    }
    """
    ref_trigger_offset, ref_horz_offset = oscilliscope_reference_reader.get_segment_times()
    _, mcp_horz_offset = mcp_reader.get_segment_times()
    _, clock_horz_offset = clock_reader.get_segment_times()
    return ak.Array({
        "i_evt": list(range(len(clock_reader.x))),
        "segment_time": ref_trigger_offset,
        "channel": np.stack(
            [oscilliscope_reference_reader.y, 
             mcp_reader.y, 
             clock_reader.y], 
            axis=1),
        "time": clock_reader.x,
        "timeoffsets": np.stack(
            [ref_horz_offset-ref_horz_offset, 
             mcp_horz_offset-ref_horz_offset, 
             clock_horz_offset-ref_horz_offset], 
            axis=1)
    })

generate_old_awk_array(
    ref_channel, mcp_data, clock_data
)

mcp_fit_data = mcp.fit(
    old, 
    dat2root_path="/home/users/hswanson13/ETL_TestingDAQ/oscilliscope_fitting/NetScopeStandalone_Dat2Root_MCP_FITTER", 
    config_path="/home/users/hswanson13/ETL_TestingDAQ/oscilliscope_fitting/LecroyScope_v12.config"
)

/home/users/hswanson13/ETL_TestingDAQ/oscilliscope_fitting/NetScopeStandalone_Dat2Root_MCP_FITTER: error while loading shared libraries: libThread.so: cannot open shared object file: No such file or directory


CalledProcessError: Command '/home/users/hswanson13/ETL_TestingDAQ/oscilliscope_fitting/NetScopeStandalone_Dat2Root_MCP_FITTER --correctForTimeOffsets --input_file=/tmp/input_dat2root_temp.root --output_file=/tmp/output_dat2root_temp.root --config=/home/users/hswanson13/ETL_TestingDAQ/oscilliscope_fitting/LecroyScope_v12.config --save_meas' returned non-zero exit status 127.

In [26]:
old = uproot.open(
"/home/users/hswanson13/ETL_TestingDAQ/unit_test/asserted_output/run_5100/output_run_5100_rb0.root")["pulse"].arrays()

old.type.show()

5000 * {
    event: int32,
    l1counter: int32,
    row: var * int32,
    col: var * int32,
    tot_code: var * float32,
    toa_code: var * float32,
    cal_code: var * float32,
    elink: var * float32,
    crc: var * int32,
    chipid: var * int32,
    bcid: int32,
    counter_a: var * int32,
    nhits: int32,
    nhits_trail: var * int32
}


In [28]:
import numpy as np
a = np.array([1,2,3,4,5,6,7,8,9,10])
b = np.array([1,2,3,4,5,6,7,8,9,10])
c = np.array([1,2,3,4,5,6,7,8,9,10])
d = np.array([1,2,3,4,5,6,7,8,9,10])

np.stack([a,b,c,d], axis=1)

array([[ 1,  1,  1,  1],
       [ 2,  2,  2,  2],
       [ 3,  3,  3,  3],
       [ 4,  4,  4,  4],
       [ 5,  5,  5,  5],
       [ 6,  6,  6,  6],
       [ 7,  7,  7,  7],
       [ 8,  8,  8,  8],
       [ 9,  9,  9,  9],
       [10, 10, 10, 10]])